In [15]:
import pandas as pd
import gensim.models
from nltk.tokenize import word_tokenize
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')
from tqdm import tqdm

In [3]:
import spacy
nlp=spacy.load('en_core_web_sm')

In [4]:
import os
os.getcwd()

'/Users/mstudio/repository/viraltext'

In [5]:
import os
os.chdir('snippet-dedupe')

In [6]:
os.listdir()

['dedupe_two_extended_cass_county_republican_slave.csv',
 'dedupe_two_extended_edgefield_slave.csv',
 'dedupe_two_extended_anti_slavery_bugle_servant.csv',
 'dedupe_two_extended_abbeville_banner_servant.csv',
 'dedupe_two_extended_edgefield_servant.csv',
 'dedupe_two_extended_keowee_courier_servant.csv',
 'dedupe_two_extended_nashville_union_american_slave.csv',
 'dedupe_two_extended_national_era_slave.csv',
 'dedupe_two_extended_fayetteville_observer_servant.csv',
 'dedupe_two_extended_daily_dispatch_slave.csv',
 'dedupe_two_extended_texas_republican_servant.csv',
 'dedupe_two_extended_daily_dispatch_servant.csv',
 'dedupe_two_extended_green_mountain_freeman_servant.csv',
 'dedupe_two_extended_fayetteville_observer_slave.csv',
 'dedupe_two_extended_athens_post_slave.csv',
 'dedupe_two_extended_marshall_county_republican_slave.csv',
 'dedupe_two_extended_keowee_courier_slave.csv',
 'dedupe_two_extended_marshall_county_republican_servant.csv',
 'dedupe_two_extended_vermont_watchman_stat

In [7]:
south_slave=[pd.read_csv('dedupe_two_extended_abbeville_banner_slave.csv'),
pd.read_csv('dedupe_two_extended_daily_dispatch_slave.csv'),
pd.read_csv('dedupe_two_extended_nashville_union_american_slave.csv'),
pd.read_csv('dedupe_two_extended_edgefield_slave.csv')
]
north_slave=[pd.read_csv('dedupe_two_extended_anti_slavery_bugle_slave.csv'),
pd.read_csv('dedupe_two_extended_national_era_slave.csv'),
pd.read_csv('dedupe_two_extended_marshall_county_republican_slave.csv'),
pd.read_csv('dedupe_two_extended_green_mountain_freeman_slave.csv')
]

In [8]:
south_slave=pd.concat(south_slave)
north_slave=pd.concat(north_slave)

In [9]:
south_slave['context']=south_slave['context'].str.replace('slave', 'wxyz')

In [10]:
frame=[south_slave, north_slave]

pre-processing

In [11]:
df=pd.concat(frame)

In [12]:
df.size

754256

In [13]:
df_dict=df.to_dict('records')

In [17]:
def lemmatization(dataframe:pd.DataFrame()):
    dataframe['stopword']=dataframe['context'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop)]))
    dataframe['punct']=dataframe['stopword'].str.replace('[^\w\s]', '')
    dataframe['lower']=dataframe['punct'].str.lower()
    dataframe['lemma']=dataframe['lower'].apply(lambda row:' '.join([w.lemma_ for w in nlp(row)]))
    dataframe['token']=dataframe['lemma'].apply(word_tokenize)
    return dataframe

In [22]:
def remove_stop_words(sentence):
    stopwords=stop
    word_list=sentence.split()
    clean_sentence=' '.join([word for word in word_list if word not in stopwords])
    return clean_sentence

In [45]:
%%time
stopword=[]
punct=[]
lower=[]
lemma=[]
token=[]
for row in tqdm(df_dict):
    word_list=row['context'].split()
    clean_sentence=' '.join([word for word in word_list if word not in stop])
    stopword.append(clean_sentence)
    punct.append(clean_sentence.replace('[^\w\s]', ''))
    lower.append(clean_sentence.lower())
    lemma.append(' '.join([w.lemma_ for w in nlp(clean_sentence.lower())]))
    token.append(word_tokenize(' '.join([w.lemma_ for w in nlp(clean_sentence.lower())])))

100%|██████████| 188564/188564 [30:15<00:00, 103.89it/s]

CPU times: user 30min 10s, sys: 8.03 s, total: 30min 18s
Wall time: 30min 15s


In [46]:
df['stopword']=stopword
df['punct']=punct
df['lower']=lower
df['lemma']=lemma
df['token']=token

word2vec

In [47]:
from gensim.models.word2vec import Word2Vec

In [48]:
model=Word2Vec([row for row in df['token']], min_count=10, workers=3, window=5, sg=1)

In [49]:
from gensim.models import KeyedVectors
south_north_slave=model.wv.most_similar(positive=['wxyz'], negative=['slave'], topn=100)
north_south_slave=model.wv.most_similar(positive=['slave'], negative=['wxyz'], topn=100)

In [50]:
south_north_slave

[('aad', 0.31551313400268555),
 ('waa', 0.2840401530265808),
 ('~', 0.26450347900390625),
 ('»', 0.24876858294010162),
 ('«', 0.23075219988822937),
 ('—', 0.22763964533805847),
 ('■', 0.2217412143945694),
 ('af', 0.21829475462436676),
 ('alave', 0.2170083075761795),
 ('ne', 0.2132846713066101),
 ('eat', 0.21217074990272522),
 ('ef', 0.210989311337471),
 ('wxyzry', 0.20771364867687225),
 ('assault', 0.2024698406457901),
 ('thte', 0.20214338600635529),
 ('la', 0.2017408013343811),
 ('oa', 0.19961397349834442),
 ('blacksmith', 0.19945819675922394),
 ('shop', 0.19914406538009644),
 ('wae', 0.19463858008384705),
 ('te', 0.19305093586444855),
 ('tat', 0.1925216168165207),
 ('tea', 0.1915876865386963),
 ('tba', 0.1884196400642395),
 ('_', 0.18687118589878082),
 ('fat', 0.18282391130924225),
 ('#', 0.18046654760837555),
 ('oat', 0.17704318463802338),
 ('k', 0.17596516013145447),
 ('arraign', 0.1741638034582138),
 ('tae', 0.17153820395469666),
 ('asd', 0.17105920612812042),
 ('carpenter', 0.170

In [51]:
north_south_slave

[('wero', 0.4625748097896576),
 ('nro', 0.45808741450309753),
 ('nre', 0.43329063057899475),
 ('nnd', 0.4209197461605072),
 ('llio', 0.42042529582977295),
 ('tho', 0.41263535618782043),
 ('tlio', 0.40424004197120667),
 ('aro', 0.3994278609752655),
 ('theso', 0.3761707544326782),
 ('und', 0.3727759122848511),
 ('hnvo', 0.3654331862926483),
 ('hnve', 0.36238226294517517),
 ('iho', 0.3557157814502716),
 (',', 0.35446420311927795),
 ('thnt', 0.3526664078235626),
 ('thoso', 0.3517763018608093),
 ('catcher', 0.3496885597705841),
 ('wcro', 0.34880173206329346),
 ('holding', 0.3483321964740753),
 ('uud', 0.3469831347465515),
 ('whoso', 0.3468739688396454),
 ('arc', 0.34497418999671936),
 ('n', 0.34438881278038025),
 ('official', 0.34195849299430847),
 ('hns', 0.34083133935928345),
 ('sluve', 0.3383333683013916),
 ('mid', 0.336462140083313),
 ('ull', 0.33514800667762756),
 ('nil', 0.3305119276046753),
 ('nml', 0.32823190093040466),
 ('.', 0.3265268802642822),
 ('havo', 0.3252871036529541),
 ('c

In [14]:
dataframe=pd.DataFrame(slave_servant, columns=('token', 'similarity'))
outname='slave_servant_'+folder+'_min10.csv'
outdir='../word2vec-dedupe'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname=os.path.join(outdir, outname)
dataframe.to_csv(fullname)

In [15]:
dataframe=pd.DataFrame(servant_slave, columns=('token', 'similarity'))
outname='servant_slave_'+folder+'_min10.csv'
outdir='../word2vec-dedupe'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname=os.path.join(outdir, outname)
dataframe.to_csv(fullname)